------
# **Dementia Patients -- Analysis and Prediction**
### ***Author : Akhilesh Vyas***
### ****Date : August, 2019****



# ***Result Plots***

- <a href='#00'>0. Setup </a>
    - <a href='#00.1'>0.1. Load libraries </a>
    - <a href='#00.2'>0.2. Define paths </a>

- <a href='#01'>1. Data Preparation </a>  
    - <a href='#01.1'>1.1. Read Data </a> 
    - <a href='#01.2'>1.2. Prepare data  </a>
    - <a href='#01.3'>1.3. Prepare target </a>
    - <a href='#01.4'>1.4. Removing Unwanted Features </a>
    
- <a href='#02'>2. Data Analysis</a> 
    - <a href='#02.1'>2.1. Feature </a> 
    - <a href='#02.2'>2.2. Target </a> 
    
- <a href='#03'>3. Data Preparation and Vector Transformation</a>

- <a href='#04'>4. Analysis and Imputing Missing Values </a>

- <a href='#05'>5. Feature Analysis</a> 
    - <a href='#05.1'>5.1. Correlation Matrix</a>
    - <a href='#05.2'>5.2. Feature and target </a>
    - <a href='#05.3'>5.3. Feature Selection Models </a>
    
- <a href='#06'>6.Machine Learning -Classification Model</a> 

# <a id='00'>0. Setup </a>

# <a id='00.1'>0.1 Load libraries </a>

Loading Libraries

In [9]:
import sys
sys.path.insert(1, '../preprocessing/')
import numpy as np
import pickle
import scipy.stats as spstats
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from sklearn.datasets.base import Bunch
from data_transformation_cls import FeatureTransform
from ast import literal_eval
import plotly.figure_factory as ff
import plotly.offline as py
import plotly.graph_objects as go

import pandas as pd
pd.set_option('display.max_columns', None)  
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

from ordered_set import OrderedSet

%matplotlib inline

# <a id='00.2'>0.2 Define paths </a>

In [10]:
# data_path
data_path = '../../../datalcdem/data/optima/dementia_18July/data_kernel_sl_nr_ft_newfeaturevariable_maya/'
result_path = '../../../datalcdem/data/optima/dementia_18July/data_kernel_sl_nr_ft_newfeaturevariable_maya/results/'

# <a id='1'>1. Data Preparation </a> 

## <a id='01.1'>1.1. Read Data</a>

In [11]:
# Reading Data

#patients data
patient_df = pd.read_csv(data_path+'patients.csv')
print (patient_df.dtypes)
# change dataType if there is something
for col in patient_df.columns: 
    if 'Date' in col:
        patient_df[col] = pd.to_datetime(patient_df[col]) 
 
display(patient_df.head(5))

patient_id                   int64  
gender                       object 
dementia                     bool   
smoker                       object 
alcohol                      object 
education                    object 
bmi                          float64
weight                       float64
ageAtFirstEpisode            int64  
ageAtLastEpisode             int64  
malignantComorbidity         object 
systemicComorbidity          object 
cardiovascularComorbidity    bool   
metabolicComorbidity         bool   
infectiousComorbidity        bool   
neoplasticComorbidity        bool   
respiratoryComorbidity       bool   
neurologicalComorbidity      bool   
inflammatoryComorbidity      bool   
apoe                         object 
epDateInicial                object 
mmseInicial                  int64  
mmseInicialDiscretized       object 
epDateFinal                  object 
mmseFinal                    int64  
mmseFinalDiscretized         object 
epDateToSevereData           object 
e

,patient_id,gender,dementia,smoker,alcohol,education,bmi,weight,ageAtFirstEpisode,ageAtLastEpisode,malignantComorbidity,systemicComorbidity,cardiovascularComorbidity,metabolicComorbidity,infectiousComorbidity,neoplasticComorbidity,respiratoryComorbidity,neurologicalComorbidity,inflammatoryComorbidity,apoe,epDateInicial,mmseInicial,mmseInicialDiscretized,epDateFinal,mmseFinal,mmseFinalDiscretized,epDateToSevereData,epDateToMildData,epDateToModerateData
0,1,Male,False,smoker,mild_drinking,medium,28.0,81.0,71,80,False,False,False,False,False,False,False,False,False,E3E3,1998-01-13,30,normal,2007-05-04,30,normal,NaT,NaT,NaT
1,14,Female,False,no_smoker,mild_drinking,medium,19.0,48.0,79,88,False,True,True,True,True,False,False,False,False,E3E4,1990-03-26,29,normal,1999-03-19,30,normal,NaT,NaT,NaT
2,16,Male,False,no_smoker,extreme_drinking,higher,NaN,NaN,78,84,False,False,True,False,False,False,False,False,False,E3E4,1991-07-15,28,normal,1996-10-21,26,normal,NaT,1995-07-03,NaT
3,19,Male,True,smoker,extreme_drinking,higher,NaN,NaN,56,61,True,True,False,True,False,False,False,False,False,E3E4,1989-09-27,12,severe,1994-07-06,-1,severe,1989-09-27,NaT,1990-03-27
4,32,Male,False,no_smoker,mild_drinking,higher,NaN,NaN,72,73,False,False,False,False,False,False,False,False,False,E3E3,1990-05-04,15,moderate,1991-03-26,-1,severe,NaT,NaT,1990-05-04


In [12]:
# From API Data- comorbidities and treatments

# Treatment data
treatmnt_df = pd.read_csv(data_path+'Treatments.csv')
display(treatmnt_df.head(5))
treatmnt_dict = dict(zip(treatmnt_df['name'], treatmnt_df['CUI_ID']))
print ('\n Unique Treatment data size: {}\n'.format(len(treatmnt_dict)))

# Patient treatment data
patient_treatmnt_df = pd.read_csv(data_path+'patient_Treatments.csv')
patient_treatmnt_df['drugName_cui'] = patient_treatmnt_df['drugName'].apply(lambda x: treatmnt_dict[x])
# patient_treatmn_list_df = patient_treatmnt_df.groupby(by='patient_id', as_index=False).agg(lambda x: x.tolist()) # equivalent to below\n",
patient_treatmn_list_df = patient_treatmnt_df.groupby(by='patient_id', as_index=False).agg(lambda x: x.tolist())
display(patient_treatmn_list_df.head(5))

# Patient comorbidities data list
patient_treatmn_list = patient_treatmnt_df['drugName_cui'].unique()
print ('\n Unique Treatment data size: {}, Any NULL Value: {}\n'.format(patient_treatmn_list.size, pd.isnull(patient_treatmn_list).any()))

# Comorbidities data
comorb_df = pd.read_csv(data_path+'comorbidities.csv')
display(comorb_df.head(5))
comorb_dict = dict(zip(comorb_df['name'], comorb_df['CUI_ID']))
print ('\n Unique Comorbidities data size: {}\n'.format(len(comorb_dict)))

# Patient comorbidities data
patient_combor_df = pd.read_csv(data_path+'patient_Comorbidities.csv')
patient_combor_df['disorderName_cui'] = patient_combor_df['disorderName'].apply(lambda x: comorb_dict[x])
patient_combor_list_df = patient_combor_df.groupby(by=['patient_id'], as_index=False).agg(lambda x: x.tolist())
display(patient_combor_list_df.head(5))

# Patient comorbidities data list
patient_combor_list = patient_combor_df['disorderName_cui'].unique()
print ('\n Unique Comorbidities data size: {}, Any NULL Value: {}\n'.format(patient_combor_list.size, pd.isnull(patient_combor_list).any()))

,name,CUI_ID,DBpediaURL
0,Atenolol,C0004147,http://dbpedia.org/resource/Atenolol
1,Calcium_supplements,C3540037,http://dbpedia.org/resource/Creatine_supplements
2,Gliclazide,C0017631,http://dbpedia.org/resource/Gliclazide
3,Diltiazem,C0012373,http://dbpedia.org/resource/Diltiazem
4,Beconase,C0591154,http://dbpedia.org/resource/Beclometasone_dipropionate



 Unique Treatment data size: 213



,patient_id,drugName,drugName_cui
0,1,[Atenolol],[C0004147]
1,14,"[Atenolol, Calcium_supplements, Gliclazide, Diltiazem]","[C0004147, C3540037, C0017631, C0012373]"
2,16,"[Beconase, Acetaminophen, Burinex, Aquacobalamin, Hydroxocobalamin]","[C0591154, C0000970, C0591205, C0003663, C0020316]"
3,19,[Tolbutamide],[C0040374]
4,36,[Magnesium_cation],[C2346927]



 Unique Treatment data size: 213, Any NULL Value: False



,name,CUI_ID,DBpediaURL
0,Diphtheria,C0012546,http://dbpedia.org/resource/Diphtheria
1,Diabetes,C0011847,http://dbpedia.org/resource/Diabetes_mellitus
2,Heart_failure,C0018801,http://dbpedia.org/resource/Heart_failure
3,Transient_ischemic_attack,C0007787,http://dbpedia.org/resource/Transient_ischemic_attack
4,Right_sided_cerebral_hemisphere_cerebrovascular_accident,C0586324,http://dbpedia.org/resource/Stroke



 Unique Comorbidities data size: 84



,patient_id,disorderName,disorderName_cui
0,14,"[Diphtheria, Diabetes, Heart_failure, Transient_ischemic_attack]","[C0012546, C0011847, C0018801, C0007787]"
1,16,"[Right_sided_cerebral_hemisphere_cerebrovascular_accident, Transient_ischemic_attack]","[C0586324, C0007787]"
2,19,[Diabetes],[C0011847]
3,54,[Mitral_valve_prolapse_syndrome],[C0026267]
4,99,"[Bronchiectasis, Myocardial_infarction, Diphtheria]","[C0006267, C0027051, C0012546]"



 Unique Comorbidities data size: 84, Any NULL Value: False



In [13]:
# Patient Comorbidities data
patient_com_raw_df = pd.read_csv(data_path + 'optima_patients_comorbidities.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Comorbidity_cui']]
display(patient_com_raw_df.head(5))
patient_com_raw_df['EPISODE_DATE'] = pd.to_datetime(patient_com_raw_df['EPISODE_DATE'])


# Patient Treatment data
patient_treat_raw_df = pd.read_csv(data_path + 'optima_patients_treatments.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Medication_cui']]
display(patient_treat_raw_df.head(5))
patient_treat_raw_df['EPISODE_DATE'] = pd.to_datetime(patient_treat_raw_df['EPISODE_DATE'])

# Join Patient Treatment and Comorbidities data
patient_com_treat_raw_df = pd.merge(patient_com_raw_df, patient_treat_raw_df,on=['patient_id', 'EPISODE_DATE'], how='outer')
patient_com_treat_raw_df.sort_values(by=['patient_id', 'EPISODE_DATE'],axis=0, inplace=True, ascending=True)
patient_com_treat_raw_df.reset_index(drop=True, inplace=True)

patient_com_treat_raw_df['duration_com_treat(years)'] = patient_com_treat_raw_df.groupby(by='patient_id')['EPISODE_DATE'].transform(lambda x : (x - x.iloc[0])/(np.timedelta64(1, 'D')*365.25))


featuretransform = FeatureTransform(patient_com_treat_raw_df)
classes_list = [list(featuretransform.extract_value_frm_list(patient_com_treat_raw_df[col].dropna().tolist())) 
                for col in patient_com_treat_raw_df.columns if 'Comorbidity_cui' in col or 'Medication_cui' in col]
columns_list = [col for col in patient_com_treat_raw_df.columns if 'Comorbidity_cui' in col or 'Medication_cui' in col]

patient_com_treat_raw_df = [featuretransform.apply_feature_transform(column_name=col, func_type='multiLabel', classes=clss, data_type_func='ordinal', literal=False) 
 for col, clss in zip(columns_list, classes_list)][-1]

patient_com_treat_raw_df.drop(columns=[col for col in patient_com_treat_raw_df.columns if 'Comorbidity_cui_TFV_' in col or 'Medication_cui_TFV_' in col], inplace=True)
display(patient_com_treat_raw_df.head(5))

patient_ep_comtf_treattf_dur = patient_com_treat_raw_df.drop(columns=['Comorbidity_cui', 'Medication_cui'])

display(patient_ep_comtf_treattf_dur.head(5))


#know conditionofPatient after 1 year


,patient_id,EPISODE_DATE,Comorbidity_cui
0,14,1992-04-28,[C0012546]
1,14,1999-03-19,"[C0011847, C0018801, C0007787]"
2,16,1991-07-15,"[C0586324, C0007787]"
3,19,1990-09-17,[C0011847]
4,19,1991-11-12,[C0011847]


,patient_id,EPISODE_DATE,Medication_cui
0,1,1998-01-13,[C0004147]
1,1,1998-12-14,[C0004147]
2,1,2000-01-20,[C0004147]
3,14,1994-04-14,[C0004147]
4,14,1995-04-27,[C0004147]


/home/vyasa/pythonEnv/lib/python3.6/site-packages/sklearn/preprocessing/_label.py:987: UserWarning:

unknown class(es) ['[', ']'] will be ignored



,patient_id,EPISODE_DATE,Comorbidity_cui,Comorbidity_cui_TF_,Medication_cui,Medication_cui_TF_,duration_com_treat(years)
0,1,1998-01-13,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[C0004147],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.000000
1,1,1998-12-14,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[C0004147],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.917808
2,1,2000-01-20,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[C0004147],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",2.019178
3,14,1992-04-28,[C0012546],"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.000000
4,14,1994-04-14,NaN,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",[C0004147],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1.961644


,patient_id,EPISODE_DATE,Comorbidity_cui_TF_,Medication_cui_TF_,duration_com_treat(years)
0,1,1998-01-13,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.000000
1,1,1998-12-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.917808
2,1,2000-01-20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",2.019178
3,14,1992-04-28,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",0.000000
4,14,1994-04-14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]",1.961644


In [14]:
#Preparation Features from Raw data

# Patient Comorbidities data
'''patient_com_raw_df = pd.read_csv(data_path + 'optima_patients_comorbidities.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Comorbidity_cui']]
display(patient_com_raw_df.head(5))
patient_com_raw_df['EPISODE_DATE'] = pd.to_datetime(patient_com_raw_df['EPISODE_DATE'])


# Patient Treatment data
patient_treat_raw_df = pd.read_csv(data_path + 'optima_patients_treatments.csv').groupby(by=['patient_id', 'EPISODE_DATE'], as_index=False).agg(lambda x: x.tolist())[['patient_id', 'EPISODE_DATE', 'Medication_cui']]
display(patient_treat_raw_df.head(5))
patient_treat_raw_df['EPISODE_DATE'] = pd.to_datetime(patient_treat_raw_df['EPISODE_DATE'])

# Join Patient Treatment and Comorbidities data
patient_com_treat_raw_df = pd.merge(patient_com_raw_df, patient_treat_raw_df,on=['patient_id', 'EPISODE_DATE'], how='outer')
patient_com_treat_raw_df.sort_values(by=['patient_id', 'EPISODE_DATE'],axis=0, inplace=True, ascending=True)
patient_com_treat_raw_df.reset_index(drop=True, inplace=True)
patient_com_treat_raw_df.head(5)


#Saving data
patient_com_treat_raw_df.to_csv(data_path + 'patient_com_treat_episode_df.csv', index=False)'''

# Extracting selected features from Raw data
sel_col_df = pd.read_excel(data_path+'Variable_Guide_Highlighted_Fields_.xlsx')
display(sel_col_df.head(5))
sel_cols = [i+j.replace('+', ':')for i,j in zip(sel_col_df['Sub Category '].tolist(), sel_col_df['Variable Label'].tolist())]
rem_cols= ['OPTIMA DIAGNOSES V 2010: OTHER SYSTEMIC ILLNESS: COMMENT'] # Missing column in the dataset
sel_cols = sorted(list(set(sel_cols)-set(rem_cols)))
print (sel_cols)

columns_selected = list(OrderedSet(['GLOBAL_PATIENT_DB_ID', 'EPISODE_DATE', 'Age At Episode', 'CAMDEX SCORES: MINI MENTAL SCORE', 
                    'OPTIMA DIAGNOSES V 2010: PETERSEN MCI', 'OPTIMA DIAGNOSES V 2010: PETERSEN MCI TYPE',
                    'DIAGNOSIS 334-351: PRIMARY PSYCHIATRIC DIAGNOSES', 'OPTIMA DIAGNOSES V 2010: AD (NINCDS-ADSDA)'] + sel_cols))

df_datarequest = pd.read_excel(data_path+'../../optima_excel/' +'Data_Request_Jan_2019_final.xlsx')
display(df_datarequest.head(1))
df_datarequest_features = df_datarequest[columns_selected]
display(df_datarequest_features.columns)

def rename_columns(col_list):
    d = {}
    for i in col_list:
        if i=='GLOBAL_PATIENT_DB_ID':
            d[i]='patient_id'
        elif 'CAMDEX SCORES: ' in i:
            d[i]=i.replace('CAMDEX SCORES: ', '').replace(' ', '_')
        elif 'CAMDEX ADMINISTRATION 1-12: ' in i:
            d[i]=i.replace('CAMDEX ADMINISTRATION 1-12: ', '').replace(' ', '_')
        elif 'DIAGNOSIS 334-351: ' in i:
            d[i]=i.replace('DIAGNOSIS 334-351: ', '').replace(' ', '_')
        elif 'OPTIMA DIAGNOSES V 2010: ' in i:
            d[i]=i.replace('OPTIMA DIAGNOSES V 2010: ', '').replace(' ', '_')
        elif 'PM INFORMATION: ' in i:
            d[i]=i.replace('PM INFORMATION: ', '').replace(' ', '_')
        else:
            d[i]=i.replace(' ', '_')
    return d
    
df_datarequest_features.rename(columns=rename_columns(df_datarequest_features.columns.tolist()), inplace=True)

display(df_datarequest_features.head(5))
# df_datarequest_features.drop(columns=['Age_At_Episode', 'PETERSEN_MCI_TYPE'], inplace=True)
display(df_datarequest_features.head(5))

# drop columns having out of range MMSE value
# df_datarequest_features = df_datarequest_features[(df_datarequest_features['MINI_MENTAL_SCORE']<=30) & (df_datarequest_features['MINI_MENTAL_SCORE']>=0)]

# Merging Join Patient Treatment, Comorbidities and selected features from raw data
#patient_com_treat_raw_df['EPISODE_DATE'] = pd.to_datetime(patient_com_treat_raw_df['EPISODE_DATE'])
#patient_com_treat_fea_raw_df = pd.merge(patient_com_treat_raw_df,df_datarequest_features,on=['patient_id', 'EPISODE_DATE'], how='left')
#patient_com_treat_fea_raw_df.sort_values(by=['patient_id', 'EPISODE_DATE'],axis=0, inplace=True, ascending=True)
#patient_com_treat_fea_raw_df.reset_index(inplace=True, drop=True)
#display(patient_com_treat_fea_raw_df.head(5))
patient_com_treat_fea_raw_df = df_datarequest_features # Need to be changed ------------------------

# Filling misssing MMSE value with patient group Average

#patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE']\
#                                        = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['MINI_MENTAL_SCORE'].transform(lambda x: x.fillna(x.mean()))
display(patient_com_treat_fea_raw_df.head(5))

#  19<=Mild<=24 , 14<=Moderate<=18 , Severe<=13 
patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY']=np.nan

def change_minimentalscore_to_category(df):
    df.loc[(df['MINI_MENTAL_SCORE']<=30) & (df['MINI_MENTAL_SCORE']>24),'MINI_MENTAL_SCORE_CATEGORY'] = 'Normal'
    df.loc[(df['MINI_MENTAL_SCORE']<=24) & (df['MINI_MENTAL_SCORE']>=19),
           'MINI_MENTAL_SCORE_CATEGORY'] = 'Mild'
    df.loc[(df['MINI_MENTAL_SCORE']<=18) & (df['MINI_MENTAL_SCORE']>=14),
           'MINI_MENTAL_SCORE_CATEGORY'] = 'Moderate'
    df.loc[(df['MINI_MENTAL_SCORE']<=13) & (df['MINI_MENTAL_SCORE']>=0),'MINI_MENTAL_SCORE_CATEGORY'] = 'Severe'
    
    return df

patient_com_treat_fea_raw_df = change_minimentalscore_to_category(patient_com_treat_fea_raw_df)

# saving file
patient_com_treat_fea_raw_df.to_csv(data_path + 'patient_com_treat_fea_episode_raw_without_expand_df.csv', index=False)

# Set line number for treatment line
def setLineNumber(lst):
    lst_dict = {ide:0 for ide in lst}
    lineNumber_list = []
    
    for idx in lst:
        if idx in lst_dict:
           lst_dict[idx] = lst_dict[idx] + 1 
           lineNumber_list.append(lst_dict[idx])
            
    return lineNumber_list

patient_com_treat_fea_raw_df['lineNumber'] = setLineNumber(patient_com_treat_fea_raw_df['patient_id'].tolist())
display(patient_com_treat_fea_raw_df.head(5))

# Extend episode data into columns
def extend_episode_data(df):
    id_dict = {i:0 for i in df['patient_id'].tolist()}
    for x in df['patient_id'].tolist():
        if x in id_dict:
            id_dict[x]=id_dict[x]+1
    
    line_updated = [int(j) for i in id_dict.values() for j in range(1,i+1)]
    # print (line_updated[0:10])
    df.update(pd.Series(line_updated, name='lineNumber'),errors='ignore')
    print ('\n----------------After creating line-number for each patients------------------')
    display(df.head(20))
    
    # merging episodes based on id and creating new columns for each episode
    r = df['lineNumber'].max()
    print ('Max line:',r)
    l = [df[df['lineNumber']==i] for i in range(1, int(r+1))]
    print('Number of Dfs to merge: ',len(l))
    df_new = pd.DataFrame()
    tmp_id = []
    for i, df_l in enumerate(l):
        df_l = df_l[~df_l['patient_id'].isin(tmp_id)]
        for j, df_ll in enumerate(l[i+1:]):
            #df_l = df_l.merge(df_ll, on='id', how='left', suffix=(str(j), str(j+1))) #suffixe is not working
            #print (j)
            df_l = df_l.join(df_ll.set_index('patient_id'), on='patient_id', rsuffix='_'+str(j+1))
        tmp_id = tmp_id + df_l['patient_id'].tolist()
        #display(df_l)
        df_new = df_new.append(df_l, ignore_index=True, sort=False)
        return df_new
    


patient_com_treat_fea_raw_df['lineNumber'] = setLineNumber(patient_com_treat_fea_raw_df['patient_id'].tolist())
# drop rows with duplicated episode for a patient
patient_com_treat_fea_raw_df = patient_com_treat_fea_raw_df.drop_duplicates(subset=['patient_id', 'EPISODE_DATE'])
patient_com_treat_fea_raw_df.sort_values(by=['patient_id', 'EPISODE_DATE'], inplace=True)
columns = patient_com_treat_fea_raw_df.columns.tolist()
patient_com_treat_fea_raw_df = patient_com_treat_fea_raw_df[columns[0:2]+columns[-1:]
                                                            +columns[2:4]+columns[-2:-1]
                                                            +columns[4:-2]]
# Expand patient 
#patient_com_treat_fea_raw_df = extend_episode_data(patient_com_treat_fea_raw_df)
display(patient_com_treat_fea_raw_df.head(2))


#Saving extended episode of each patients
#patient_com_treat_fea_raw_df.to_csv(data_path + 'patient_com_treat_fea_episode_raw_df.csv', index=False)



,Category,Sub Category,Variable Label,Description,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaN,CAMDEX ADMINISTRATION 1-12,+ EST SEVERITY OF DEMENTIA,"Data Type: Coded value, Integer, see list of code-decode pairs:",0 = None,1 = Minimal,2 = Mild,3 = Moderate,4 = Severe,8 = Doesn,9 = Not asked,NaN
1,NaN,CAMDEX ADMINISTRATION 1-12,+ EST OF SEVERITY OF DEPRESSION,"Data Type: Coded value, Integer, see list of code-decode pair:",0 = None,1 = Minimal,2 = Mild,3 = Moderate,4 = Severe,8 = Doesn,9 = Not asked,NaN
2,NaN,DIAGNOSIS 334-351,+ PRIMARY PSYCHIATRIC DIAGNOSES,"Data Type: Coded value, Integer, see list of code-decode pairs",0 = Absent,1 = Present,9 = Not Asked,NaN,NaN,NaN,NaN,NaN
3,NaN,DIAGNOSIS 334-351,+ DEPRESSIVE ILLNESS,"Data Type: Coded value, Integer, see list of code-decode pairs",0 = Absent,1 = Present,9 = Not Asked,NaN,NaN,NaN,NaN,NaN
4,NaN,DIAGNOSIS 334-351,+ SECONDARY DEMENTIA,"Data Type: Coded value, Integer, see list of code-decode pairs",0 = Absent,1 = Present,9 = Not Asked,NaN,NaN,NaN,NaN,NaN


['CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION', 'CAMDEX ADMINISTRATION 1-12: EST SEVERITY OF DEMENTIA', 'DIAGNOSIS 334-351: ANXIETY/PHOBIC', 'DIAGNOSIS 334-351: DEPRESSIVE ILLNESS', 'DIAGNOSIS 334-351: PARANOID/PARAPHRENIC', 'DIAGNOSIS 334-351: PRIMARY PSYCHIATRIC DIAGNOSES', 'DIAGNOSIS 334-351: SECONDARY DEMENTIA', 'OPTIMA DIAGNOSES V 2010: AD (NINCDS-ADSDA)', 'OPTIMA DIAGNOSES V 2010: CERBRO-VASCULAR DISEASE PRESENT', 'OPTIMA DIAGNOSES V 2010: CEREBRO-VASCULAR RISK FACTORS', 'OPTIMA DIAGNOSES V 2010: COGNITIVE IMPAIRMENT', 'OPTIMA DIAGNOSES V 2010: CORTICO-BASAL DEGENERATION', 'OPTIMA DIAGNOSES V 2010: DEMENTIA OTHER', 'OPTIMA DIAGNOSES V 2010: DEMENTIA PRESENT', 'OPTIMA DIAGNOSES V 2010: DIAGNOSES COMMENT', 'OPTIMA DIAGNOSES V 2010: DIAGNOSTIC CODE', 'OPTIMA DIAGNOSES V 2010: FRONTO-TEMPORAL DEMENTIA', 'OPTIMA DIAGNOSES V 2010: LEWY-BODY DISEASE', 'OPTIMA DIAGNOSES V 2010: LEWY-BODY DISEASE SEVERITY', 'OPTIMA DIAGNOSES V 2010: MIXED DEMENTIA', 'OPTIMA DIAGNOSES V 2010:

,GLOBAL_PATIENT_DB_ID,STUDY_ID,STUDY,DOD,Age At Episode,GENDER,EPISODE_DATE,EPISODE,CAMDEX ADMINISTRATION 1-12: DATE OF EXAMINATION,CAMDEX ADMINISTRATION 1-12: EST SEVERITY OF DEMENTIA,CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION,CAMDEX ADMINISTRATION 1-12: VISUALLY IMPAIRED,CAMDEX ADMINISTRATION 1-12: HARD OF HEARING,CAMDEX ADMINISTRATION 1-12: TOTALLY CARED FOR,CAMDEX ADMINISTRATION 1-12: OTHER SIGNIFICANT IMPAIRMENT,CAMDEX ADMINISTRATION 1-12: OTHER SIGNIFICANT IMPAIRMENT: TEXT,PRESENT STATE 13-46: PRESENT STATE 13-46,PRESENT STATE 13-46: (013) NAME KNOWN,PRESENT STATE 13-46: (014) AGE KNOWN,PRESENT STATE 13-46: (015) DATE OF BIRTH KNOWN,PRESENT STATE 13-46: (016) AGE LEFT SCHOOL,PRESENT STATE 13-46: (017) YEARS IN FURTHER EDUCATION,PRESENT STATE 13-46: (017) YEARS IN FURTHER EDUCATION: TEXT,PRESENT STATE 13-46: (018) LIVING ARRANGEMENTS,PRESENT STATE 13-46: (019) HEADACHE,PRESENT STATE 13-46: (020) DIZZYNESS,PRESENT STATE 13-46: (021) FALLS,PRESENT STATE 13-46: (022) TIA,PRESENT STATE 13-46: (023) FALLS ASLEEP,PRESENT STATE 13-46: (024) WAKEFUL,PRESENT STATE 13-46: (025) WAKES EARLY,PRESENT STATE 13-46: (025) WAKES EARLY: TEXT,PRESENT STATE 13-46: (026) APPETITE CHANGE,PRESENT STATE 13-46: (027) WEIGHT CHANGE,PRESENT STATE 13-46: (027) WEIGHT CHANGE: TEXT,PRESENT STATE 13-46: (028) COPING LESS,PRESENT STATE 13-46: (029) DECISION MAKING DIFFICULT,PRESENT STATE 13-46: (030) PLEASURE LOSS,PRESENT STATE 13-46: (031) ENERGY LOSS,PRESENT STATE 13-46: (032) ALONE PREFERENCE,PRESENT STATE 13-46: (033) CONCENTRATION DIFFICULT,PRESENT STATE 13-46: (034) SLOWED SPEECH,PRESENT STATE 13-46: (035) SLOWED THOUGHT,PRESENT STATE 13-46: (036) FEELING DEPRESSED,PRESENT STATE 13-46: (037) DURATION OF DEPRESSION,PRESENT STATE 13-46: (038) REASON FOR DEPRESSION,PRESENT STATE 13-46: (038) REASON FOR DEPRESSION: TEXT,PRESENT STATE 13-46: (039) DIFFERENT FROM SADNESS,PRESENT STATE 13-46: (040) CHEERS UP,PRESENT STATE 13-46: (041) DIURNAL CHANGE,PRESENT STATE 13-46: (042) BLAMES OTHERS,PRESENT STATE 13-46: (043) WORTHLESS OR GUILTY,PRESENT STATE 13-46: (044) FUTURE FEELINGS,PRESENT STATE 13-46: (045) NOT WORTH LIVING,PRESENT STATE 13-46: (046) SUICIDAL FEELINGS,PRESENT STATE 47-73: PRESENT STATE 47-73,PRESENT STATE 47-73: (047) WORRIES MORE,PRESENT STATE 47-73: (048) IRRITABLE,PRESENT STATE 47-73: (049) ANXIOUS,PRESENT STATE 47-73: (050) PHYSICAL SYMPTOMS,PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS,PRESENT STATE 47-73: (051) ANXIOUS SITUATIONS: TEXT,PRESENT STATE 47-73: (052) PANIC ATTACKS,PRESENT STATE 47-73: (053) DURATION OF PANIC,PRESENT STATE 47-73: (054) MONEY HANDLING DIFFICULT,PRESENT STATE 47-73: (055) HOUSEHOLD TASKS DIFFICULT,PRESENT STATE 47-73: (056) BLADDER CONTROL DIFFICULT,PRESENT STATE 47-73: (057) PHYSICAL PROBLEMS,PRESENT STATE 47-73: (058) MEMORY PROBLEM,PRESENT STATE 47-73: (059) FORGETS OBJECTS,PRESENT STATE 47-73: (060) FORGETS NAMES,PRESENT STATE 47-73: (061) FORGETS WAY,PRESENT STATE 47-73: (062) DURATION OF MEMORY PROBLEMS,PRESENT STATE 47-73: (063) ONSET OF MEMORY PROBLEMS,PRESENT STATE 47-73: (064) CHANGE IN MEMORY PROBLEMS,PRESENT STATE 47-73: (065) REFERS MORE TO THE PAST,PRESENT STATE 47-73: (066) WORD FINDING DIFFICULTY,PRESENT STATE 47-73: (066) WORD FINDING DIFFICULTY: DURATION,PRESENT STATE 47-73: (068) AUDITORY HALLUCINATIONS,PRESENT STATE 47-73: (068) AUDITORY HALLUCINATIONS: DURATION,PRESENT STATE 47-73: (068) AUDITORY HALLUCINATIONS: TEXT,PRESENT STATE 47-73: (069) VISUAL HALLUCINATIONS,PRESENT STATE 47-73: (069) VISUAL HALLUCINATIONS: DURATION,PRESENT STATE 47-73: (069) VISUAL HALLUCINATIONS: TEXT,PRESENT STATE 47-73: (070) WATCHED OR SPIED ON,PRESENT STATE 47-73: (070) WATCHED OR SPIED ON: DURATION,PRESENT STATE 47-73: (071) PASSIVITY FEELINGS,PRESENT STATE 47-73: (071) PASSIVITY FEELINGS: DURATION,PRESENT STATE 47-73: (072) HYPOCHONDRIACAL DELUSIONS,PRESENT STATE 47-73: (072) HYPOCHONDRIACAL DELUSIONS: DURATION,PRESENT STATE 47-73: (073) ILLUSORY INTERFERENCE,PRESENT STATE 47-73: (073) ILLUSO

Index(['GLOBAL_PATIENT_DB_ID', 'EPISODE_DATE', 'Age At Episode',
       'CAMDEX SCORES: MINI MENTAL SCORE',
       'OPTIMA DIAGNOSES V 2010: PETERSEN MCI',
       'OPTIMA DIAGNOSES V 2010: PETERSEN MCI TYPE',
       'DIAGNOSIS 334-351: PRIMARY PSYCHIATRIC DIAGNOSES',
       'OPTIMA DIAGNOSES V 2010: AD (NINCDS-ADSDA)',
       'CAMDEX ADMINISTRATION 1-12: EST OF SEVERITY OF DEPRESSION',
       'CAMDEX ADMINISTRATION 1-12: EST SEVERITY OF DEMENTIA',
       'DIAGNOSIS 334-351: ANXIETY/PHOBIC',
       'DIAGNOSIS 334-351: DEPRESSIVE ILLNESS',
       'DIAGNOSIS 334-351: PARANOID/PARAPHRENIC',
       'DIAGNOSIS 334-351: SECONDARY DEMENTIA',
       'OPTIMA DIAGNOSES V 2010: CERBRO-VASCULAR DISEASE PRESENT',
       'OPTIMA DIAGNOSES V 2010: CEREBRO-VASCULAR RISK FACTORS',
       'OPTIMA DIAGNOSES V 2010: COGNITIVE IMPAIRMENT',
       'OPTIMA DIAGNOSES V 2010: CORTICO-BASAL DEGENERATION',
       'OPTIMA DIAGNOSES V 2010: DEMENTIA OTHER',
       'OPTIMA DIAGNOSES V 2010: DEMENTIA PRESENT',
      

,patient_id,EPISODE_DATE,Age_At_Episode,MINI_MENTAL_SCORE,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
0,1,1998-01-13,71,30.0,0.0,9.0,0.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1998-12-14,72,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2000-01-20,73,30.0,0.0,9.0,9.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2000-08-18,73,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,9.0,9.0,9.0,NaN,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2002-02-21,75,30.0,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,patient_id,EPISODE_DATE,Age_At_Episode,MINI_MENTAL_SCORE,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
0,1,1998-01-13,71,30.0,0.0,9.0,0.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1998-12-14,72,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2000-01-20,73,30.0,0.0,9.0,9.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2000-08-18,73,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,9.0,9.0,9.0,NaN,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2002-02-21,75,30.0,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,patient_id,EPISODE_DATE,Age_At_Episode,MINI_MENTAL_SCORE,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
0,1,1998-01-13,71,30.0,0.0,9.0,0.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1998-12-14,72,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2000-01-20,73,30.0,0.0,9.0,9.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2000-08-18,73,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,9.0,9.0,9.0,NaN,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2002-02-21,75,30.0,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,patient_id,EPISODE_DATE,Age_At_Episode,MINI_MENTAL_SCORE,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT,MINI_MENTAL_SCORE_CATEGORY,lineNumber
0,1,1998-01-13,71,30.0,0.0,9.0,0.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,1
1,1,1998-12-14,72,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,1,2000-01-20,73,30.0,0.0,9.0,9.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,3
3,1,2000-08-18,73,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,9.0,9.0,9.0,9.0,NaN,0.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
4,1,2002-02-21,75,30.0,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,5


,patient_id,EPISODE_DATE,lineNumber,Age_At_Episode,MINI_MENTAL_SCORE,MINI_MENTAL_SCORE_CATEGORY,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
0,1,1998-01-13,1,71,30.0,Normal,0.0,9.0,0.0,0.0,0.0,0.0,9,9,9.0,9,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1998-12-14,2,72,NaN,NaN,0.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
display(patient_com_treat_fea_raw_df.describe(include='all'))
display(patient_com_treat_fea_raw_df.info())

,patient_id,EPISODE_DATE,lineNumber,Age_At_Episode,MINI_MENTAL_SCORE,MINI_MENTAL_SCORE_CATEGORY,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
count,9584.000000,9584,9584.000000,9584.000000,8172.000000,6511,8810.000000,8810.000000,4036.000000,8810.000000,6233.000000,6233.000000,4021.0,4031.0,4035.000000,4028.0,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,117,8799.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,8810.000000,644.000000,644.000000,370.000000,220,645.0,347,645.000000,334,645.000000,141,375.000000,207,645.0,344,645.000000,645.000000,21,645.000000,444
unique,NaN,4731,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,4.0,7.0,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64,4.0,272,NaN,312,NaN,62,NaN,52,4.0,321,NaN,NaN,15,NaN,394
top,NaN,2002-02-27 00:00:00,NaN,NaN,NaN,Normal,NaN,NaN,NaN,NaN,NaN,NaN,9.0,9.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,"Cognitive impairment but no subjective complaint, so control",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ep 1,1.0,Bronchopneumonia,NaN,2006-01-01 00:00:00,NaN,Ep.1,NaN,ep 1,9.0,Normal aged brain,NaN,NaN,Brain Donated,NaN,Chest infection
freq,NaN,12,NaN,NaN,NaN,3822,NaN,NaN,NaN,NaN,NaN,NaN,4005.0,3975.0,NaN,3979.0,NaN,NaN,NaN,NaN,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,336.0,45,NaN,3,NaN,18,NaN,109,289.0,12,NaN,NaN,7,NaN,17
first,NaN,1980-10-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,2018-02-27 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,4689.451273,NaN,7.047684,75.400981,17.930494,NaN,5.900227,8.666402,15.456392,5.980136,2.659714,3.263757,NaN,NaN,8.956877,NaN,7.001930,5.965153,5.981612,5.839501,5.824064,5.915551,NaN,4.027321,5.829058,5.829058,7.510556,7.160272,8.934733,5.828604,5.979569,5.821793,7.517367,5.816799,7.327242,5.869012,7.677072,3.281056,4.425466,128.310811,NaN,NaN,NaN,4.176744,NaN,5.578295,NaN,174.437333,NaN,NaN,NaN,3.443411,5.503876,NaN,2.637209,NaN
std,2552.249607,NaN,5.520089,9.217739,12.065300,NaN,4.203817,1.542767,643.898415,4.101083,3.832946,3.580461,NaN,NaN,0.611916,NaN,3.441597,4.112206,4.050412,4.294091,4.297239,4.161284,NaN,98.038237,4.287735,4.294745,3.337617,3.608080,0.712462,4.296577,4.183610,4.298043,3.333845,4.302216,3.352618,4.253475,3.172926,2.747013,2.460077,245.798336,NaN,NaN,NaN,4.437602,NaN,4.029645,NaN,300.480472,NaN,NaN,NaN,2.990457,4.148494,NaN,3.431474,NaN
min,1.000000,NaN,1.000000,26.000000,-1.000000,NaN,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,NaN,0.000000,0.00

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9584 entries, 0 to 9594
Data columns (total 56 columns):
patient_id                                    9584 non-null int64
EPISODE_DATE                                  9584 non-null datetime64[ns]
lineNumber                                    9584 non-null int64
Age_At_Episode                                9584 non-null int64
MINI_MENTAL_SCORE                             8172 non-null float64
MINI_MENTAL_SCORE_CATEGORY                    6511 non-null object
PETERSEN_MCI                                  8810 non-null float64
PETERSEN_MCI_TYPE                             8810 non-null float64
PRIMARY_PSYCHIATRIC_DIAGNOSES                 4036 non-null float64
AD_(NINCDS-ADSDA)                             8810 non-null float64
EST_OF_SEVERITY_OF_DEPRESSION                 6233 non-null float64
EST_SEVERITY_OF_DEMENTIA                      6233 non-null float64
ANXIETY/PHOBIC                                4021 non-null object
DEPRESSIVE

None

In [16]:
profile = patient_com_treat_fea_raw_df.profile_report(title='Dementia Profiling Report', style={'full_width':True})
profile.to_file(output_file= result_path + "dementia_data_profiling_report_output_all_patients.html") 

In [57]:
patient_com_treat_fea_raw_df[(patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY']!='Normal') & (patient_com_treat_fea_raw_df['PETERSEN_MCI']==1.0)][['MINI_MENTAL_SCORE','PETERSEN_MCI']]

,patient_id,EPISODE_DATE,lineNumber,Age_At_Episode,MINI_MENTAL_SCORE,MINI_MENTAL_SCORE_CATEGORY,PETERSEN_MCI,PETERSEN_MCI_TYPE,PRIMARY_PSYCHIATRIC_DIAGNOSES,AD_(NINCDS-ADSDA),EST_OF_SEVERITY_OF_DEPRESSION,EST_SEVERITY_OF_DEMENTIA,ANXIETY/PHOBIC,DEPRESSIVE_ILLNESS,PARANOID/PARAPHRENIC,SECONDARY_DEMENTIA,CERBRO-VASCULAR_DISEASE_PRESENT,CEREBRO-VASCULAR_RISK_FACTORS,COGNITIVE_IMPAIRMENT,CORTICO-BASAL_DEGENERATION,DEMENTIA_OTHER,DEMENTIA_PRESENT,DIAGNOSES_COMMENT,DIAGNOSTIC_CODE,FRONTO-TEMPORAL_DEMENTIA,LEWY-BODY_DISEASE,LEWY-BODY_DISEASE_SEVERITY,MIXED_DEMENTIA,MIXED_DEMENTIA_TYPE,OTHER_PARKINSONIAN_SYNDROME,OTHER_SYSTEMIC_ILLNESS_AFFECTING_COGNITION,PARKINSON_DISEASE,PARKINSON_DISEASE_SEVERITY,PROGRESSIVE_SUPRA-NUCLEAR_PALSY,SMC,VASCULAR_DEMENTIA,VCI,APOE,APOE:_RESULT,BEHAVIOUR_CHANGE_DURATION,BEHAVIOUR_CHANGE_DURATION:_TEXT,CAUSE_OF_DEATH,CAUSE_OF_DEATH:_TEXT,CORONER_PM,DATE_OF_PM,LAST_NINCDS_DIAGNOSIS,LAST_NINCDS_DIAGNOSIS:_TEXT,LAST_NORMAL_DURATION,LAST_NORMAL_DURATION:_TEXT,N-PATH_DIAGNOSIS,N-PATH_DIAGNOSIS:_TEXT,PM_DIAGNOSIS,PM_DONE,PM_DONE:_TEXT,TERMINAL_ILLNESS,TERMINAL_ILLNESS:_TEXT
120,94,2005-04-22,9,82,NaN,NaN,1.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.0,1.0,0.0,0.0,0.0,NaN,0.1,0.0,0.0,0.0,9.0,9.0,0.0,8.0,0.0,0.0,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,156,2002-06-27,3,81,NaN,NaN,1.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,272,2011-11-28,4,81,-1.0,NaN,1.0,1.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,NaN,0.2,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,397,2006-10-11,3,78,23.0,Mild,1.0,9.0,NaN,1.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,NaN,0.1,1.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
524,436,2007-09-17,10,82,NaN,NaN,1.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,0.0,NaN,6.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
692,593,2004-10-11,15,76,NaN,NaN,1.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,1.0,1.0,0.0,0.0,0.0,NaN,6.0,0.0,0.0,0.0,9.0,9.0,0.0,1.0,0.0,0.0,0.0,9.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
870,805,2011-10-31,12,81,-1.0,NaN,1.0,2.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,9.0,0.0,0.0,NaN,0.1,9.0,9.0,9.0,9.0,9.0,9.0,1.0,0.0,9.0,0.0,1.0,9.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
871,805,2012-04-05,13,82,21.0,Mild,1.0,9.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,0.0,0.0,1.0,NaN,4.0,0.0,0.0,9.0,0.0,9.0,0.0,0.0,0.0,9.0,0.0,1.0,0.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
969,913,2003-06-03,5,66,NaN,NaN,1.0,9.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,8.0,1.0,1.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
970,913,2003-12-05,6,67,-1.0,NaN,1.0,9.0,NaN,0.0,9.0,9.0,NaN,NaN,NaN,NaN,8.0,1.0,1.0,0.0,0.0,0.0,NaN,1.0,0.0,0.0,0.0,9.0,9.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
patient_com_treat_fea_raw_df['durations(years)'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['EPISODE_DATE'].transform(lambda x: (x - x.iloc[0])/(np.timedelta64(1, 'D')*365.25))
# [patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY']=='Normal']
patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)'] = patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY']
patient_com_treat_fea_raw_df.loc[(patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY']=='Normal') & 
                                                  (patient_com_treat_fea_raw_df['PETERSEN_MCI']==1), 'MINI_MENTAL_SCORE_CATEGORY(+MCI)'] = 'NormalMCI'

patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)']=='NormalMCI']

patient_com_treat_fea_raw_df.head(10)

In [ ]:
# patients - time taken for changing state
patient_com_treat_fea_raw_df['State_Update(years)'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['durations(years)'].transform(lambda x: x - x.shift(+1))
patient_com_treat_fea_raw_df['MMSE_Change'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['MINI_MENTAL_SCORE'].transform(lambda x: x - x.shift(+1))
patient_com_treat_fea_raw_df['MMS_CAT_HOP'] = patient_com_treat_fea_raw_df.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[1.0,2.0,3.0,4.0,5.0]).groupby(by=['patient_id'])['MINI_MENTAL_SCORE_CATEGORY(+MCI)'].transform(lambda x: x - x.shift(+1))
patient_com_treat_fea_raw_df['MMS_CAT_HOP_NOMCI'] = patient_com_treat_fea_raw_df.replace(['Normal','Mild', 'Moderate', 'Severe'],[1.0,2.0,3.0,4.0]).groupby(by=['patient_id'])['MINI_MENTAL_SCORE_CATEGORY'].transform(lambda x: x - x.shift(+1))
patient_com_treat_fea_raw_df.loc[patient_com_treat_fea_raw_df['MMS_CAT_HOP']<=-2, 'Misdiagnosed'] = 'YES'
patient_com_treat_fea_raw_df.loc[patient_com_treat_fea_raw_df['MMS_CAT_HOP']>-2, 'Misdiagnosed'] = 'NO'
patient_com_treat_fea_raw_df.loc[patient_com_treat_fea_raw_df['MMS_CAT_HOP_NOMCI']<=-2, 'Misdiagnosed_NOMCI'] = 'YES'
patient_com_treat_fea_raw_df.loc[patient_com_treat_fea_raw_df['MMS_CAT_HOP_NOMCI']>-2, 'Misdiagnosed_NOMCI'] = 'NO'
display(patient_com_treat_fea_raw_df.tail(10))


In [ ]:
# count duration of patients
patient_total_duration = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['State_Update(years)'].agg(lambda x: x.sum()).reset_index(name='Total_Duration')
patient_total_duration.hist(column=['Total_Duration'], bins=5)

In [ ]:
#For Knowledge Graph 
patient_com_treat_fea_raw_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)_PREV']=patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['MINI_MENTAL_SCORE_CATEGORY(+MCI)'].transform(lambda x: x.shift(+1))
patient_com_treat_fea_raw_df['EPISODE_DATE_PREV'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['EPISODE_DATE'].transform(lambda x: x.shift(+1))
patient_epst_epend_misdiagnosed = patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['Misdiagnosed']=='YES'][['patient_id', 'EPISODE_DATE_PREV','EPISODE_DATE', 
                                                                                                                     'MINI_MENTAL_SCORE_CATEGORY(+MCI)_PREV','MINI_MENTAL_SCORE_CATEGORY(+MCI)','State_Update(years)', 'lineNumber']]
display(patient_epst_epend_misdiagnosed.tail(10))
patient_epst_epend_misdiagnosed.to_csv(data_path+'patient_epst_epend_misdiagnosed.csv', index=False)


patient_misdiagnosed_totaltime = patient_epst_epend_misdiagnosed.groupby(by=['patient_id'])['State_Update(years)'].agg(lambda x: x.sum()).reset_index(name='Total_Misdiagnosed_Time')
display(patient_misdiagnosed_totaltime.head(10))
patient_misdiagnosed_totaltime.to_csv(data_path+'patient_misdiagnosed_totaltime.csv', index=False)
patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['patient_id']==8735]

#AD Patients
patient_ad = patient_com_treat_fea_raw_df[(patient_com_treat_fea_raw_df['AD_STATUS']==1)|(patient_com_treat_fea_raw_df['AD_STATUS']==2)][['patient_id','EPISODE_DATE_PREV', 'EPISODE_DATE', 'AD_STATUS']]
patient_ad.to_csv(data_path+'patient_ad.csv', index=False)

In [ ]:
# Plot Number of years for changing state -- Misdiagnosed - With MCI
tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMS_CAT_HOP', 'Misdiagnosed']].dropna(how='any')

import plotly.express as px
fig = px.scatter_matrix(tmp_df,
    dimensions=["State_Update(years)", "MMS_CAT_HOP"],
    color="Misdiagnosed", symbol="Misdiagnosed")
fig.show()


fig.write_image("plot_mmshop_stateupdate_misdiagnosed.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmshop_stateupdate_misdiagnosed.html')

# Plot Number of years for changing state
tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMSE_Change', 'Misdiagnosed']].dropna(how='any')


fig = px.scatter_matrix(tmp_df,
    dimensions=["State_Update(years)", "MMSE_Change"],
    color="Misdiagnosed", symbol="Misdiagnosed")
fig.show()


fig.write_image("plot_mmschange_stateupdate_misdiagnosed.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmschange_stateupdate_misdiagnosed.html')



tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMSE_Change', 'Misdiagnosed']].dropna(how='any')

fig = px.histogram(tmp_df, x="MMSE_Change", color="Misdiagnosed", marginal="rug", # can be `box`, `violin`
                         hover_data=tmp_df.columns)
fig.show()

fig.write_image("plot_mmschange_stateupdate_misdiagnosed_hist.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmschange_stateupdate_misdiagnosed_hist.html')

In [ ]:
# Plot Number of years for changing state -- Misdiagnosed - With NO MCI
tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMS_CAT_HOP_NOMCI', 'Misdiagnosed_NOMCI']].dropna(how='any')

import plotly.express as px
fig = px.scatter_matrix(tmp_df,
    dimensions=["State_Update(years)", "MMS_CAT_HOP_NOMCI"],
    color="Misdiagnosed_NOMCI", symbol="Misdiagnosed_NOMCI")
fig.show()
fig.write_image("plot_mmshop_stateupdate_misdiagnosed_nommci.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmshop_stateupdate_misdiagnosed_nomci.html')

# Plot Number of years for changing state
tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMSE_Change', 'Misdiagnosed_NOMCI']].dropna(how='any')
fig = px.scatter_matrix(tmp_df,
    dimensions=["State_Update(years)", "MMSE_Change"],
    color="Misdiagnosed_NOMCI", symbol="Misdiagnosed_NOMCI")
fig.show()
fig.write_image("plot_mmschange_stateupdate_misdiagnosed_nomci.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmschange_stateupdate_misdiagnosed_nomci.html')



tmp_df = patient_com_treat_fea_raw_df[['State_Update(years)', 'MMSE_Change', 'Misdiagnosed_NOMCI']].dropna(how='any')
fig = px.histogram(tmp_df, x="MMSE_Change", color="Misdiagnosed_NOMCI", marginal="rug", # can be `box`, `violin`
                         hover_data=tmp_df.columns)
fig.show()
fig.write_image("plot_mmschange_stateupdate_misdiagnosed_nomci_hist.png", width=800, height=400, scale=3.0)
py.plot(fig,filename='plot_mmschange_stateupdate_misdiagnosed_nomci_hist.html')

In [ ]:
patient_com_treat_fea_raw_df['State_Update(0,2,4,6,8,10,12)'] =  patient_com_treat_fea_raw_df['State_Update(years)']//2
patient_com_treat_fea_raw_df[['State_Update(0,2,4,6,8,10,12)', 'MMS_CAT_HOP']].groupby(by=['MMS_CAT_HOP', 'State_Update(0,2,4,6,8,10,12)'])
# patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='lineNumber', values='MINI_MENTAL_SCORE_CATEGORY(+MCI)')
patient_com_treat_fea_raw_df.head(5)

In [ ]:
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'Misdiagnosed', 'durations(years)']].dropna()
#patient_dur_mci_id['Misdiagnosed'].fillna('NO', inplace=True)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='durations(years)', values='Misdiagnosed')
# patient_cat_dur_id_pivot.fillna(0, inplace=True)
# patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.replace(['NO', 'YES'],[0,100],inplace=True)
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=True)
display(patient_cat_dur_id_pivot.head(5))

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot.index.values]

line_patients = patient_cat_dur_id_pivot.columns.values

mms_values = patient_cat_dur_id_pivot.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.3, "y":0.8, "title":"Misdiagnosed: NO(0), YES(100)", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='Patient misdiagnosed during years of treatment',xaxis_nticks=37,
                         xaxis={"title": "Duration(year)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_misdiagnos_duration_patientid_heatmap.html')


# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

line_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.5, "y":0.5, "title":"Misdiagnosed: NO(0), YES(100)", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=700, height=500, title='Patient misdiagnosed during years of treatment',xaxis_nticks=37,
                         xaxis={"title": "Duration(year)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 7}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_misdiagnos_duration_patientid_heatmap_sample.png", width=600, height=500, scale=3.0)
py.plot(fig,filename='plot_misdiagnos_duration_patientid_heatmap_sample.html')


In [ ]:
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'Misdiagnosed', 'lineNumber']].dropna()
#patient_dur_mci_id['Misdiagnosed'].fillna('NO', inplace=True)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='lineNumber', values='Misdiagnosed')
# patient_cat_dur_id_pivot.fillna(0, inplace=True)
# patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.replace(['NO', 'YES'],[0,100],inplace=True)
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=True)
display(patient_cat_dur_id_pivot.head(5))

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot.index.values]

line_patients = patient_cat_dur_id_pivot.columns.values

mms_values = patient_cat_dur_id_pivot.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.3, "y":0.8, "title":"Misdiagnosed: NO(0), YES(100)", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='Patient misdiagnosed during episodes',xaxis_nticks=37,
                         xaxis={"title": "Episode Number"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_misdiagnos_episode_patientid_heatmap.html')


# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

line_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.5, "y":0.5, "title":"Misdiagnosed: NO(0), YES(100)", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=700, height=500, title='Patient misdiagnosed during episodes',xaxis_nticks=37,
                         xaxis={"title": "Episode Number"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 8}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_misdiagnos_episode_patientid_heatmap_sample.png", width=600, height=600, scale=3.0)
py.plot(fig,filename='plot_misdiagnos_episode_patientid_heatmap_sample.html')


In [ ]:
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY(+MCI)', 'lineNumber']]
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='lineNumber', values='MINI_MENTAL_SCORE_CATEGORY(+MCI)')
#patient_cat_dur_id_pivot.fillna(0, inplace=True)
patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=False)
display(patient_cat_dur_id_pivot.head(5))

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot.index.values]

line_patients = patient_cat_dur_id_pivot.columns.values

mms_values = patient_cat_dur_id_pivot.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.3, "y":0.80, "title":"MINI_MENTAL_SCORE_Category_Scale: Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100))", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='MMSE progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Episode Number"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_mmscat_episode_patientid_heatmap.html')

# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

line_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=line_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.7, "y":0.50, "title":"MMS_Scale: Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100)", 'titleside':'right',"titlefont": {"size": 10} },
        showscale=True),
        layout=go.Layout(width=700, height=500, title='MMSE progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Episode Number"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 7}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_mmscat_episode_patientid_heatmapp_sample.png", width=600, height=600, scale=3.0)
py.plot(fig,filename='plot_mmscat_episode_patientid_heatmap_sample.html')




In [ ]:
patient_com_treat_fea_raw_df['durations_round(years)']= round(patient_com_treat_fea_raw_df['durations(years)'], 4)
#patient_com_treat_fea_raw_df['durations_round(years)'] = patient_com_treat_fea_raw_df['durations(years)']
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY(+MCI)', 'durations_round(years)']]
# patient_dur_mci_id.drop_duplicates(subset=['patient_id', 'durations_round(years)'],inplace=True)
# display(patient_dur_mci_id[patient_dur_mci_id.duplicated(subset=['patient_id', 'durations_round(years)'])]['patient_id'].unique().shape)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='durations_round(years)', values='MINI_MENTAL_SCORE_CATEGORY(+MCI)')
#patient_cat_dur_id_pivot.fillna(0, inplace=True)
#patient_cat_dur_id_pivot.fillna(method='bfill' , inplace=True, axis=1)
patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=False)
patient_cat_dur_id_pivot_col = patient_cat_dur_id_pivot # patient_cat_dur_id_pivot[patient_cat_dur_id_pivot.columns.tolist()[-2:]+patient_cat_dur_id_pivot.columns.tolist()[0:-2]]
display(patient_cat_dur_id_pivot_col.iloc[400:420])

import datetime
import numpy as np
import plotly.graph_objs as go

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_col.index.values]

duration_patients = patient_cat_dur_id_pivot_col.columns.values

mms_values = patient_cat_dur_id_pivot_col.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.1, "y":0.80, "title":"MINI_MENTAL_SCORE_Category_Scale_(N/A(0),Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100))", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='MMSE progression of Patients',xaxis_nticks=48,
                         xaxis={"title": "Duration(years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_mmscat_duration_patientid_heatmap.html')


# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

duration_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.7, "y":0.50, "title":"MMS_Scale: Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100)", 'titleside':'right',"titlefont": {"size": 10} },
        showscale=True),
        layout=go.Layout(width=700, height=500, title='MMSE progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Duration (years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 7}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_mmscat_duration_patientid_heatmap_sample.png", width=600, height=600, scale=3.0)
py.plot(fig,filename='plot_mmscat_duration_patientid_heatmap_sample.html')


In [ ]:
patient_com_treat_fea_raw_df['durations_round(years)']= round(patient_com_treat_fea_raw_df['durations(years)'], 4)
#patient_com_treat_fea_raw_df['durations_round(years)'] = patient_com_treat_fea_raw_df['durations(years)']
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE', 'durations_round(years)']]
#patient_dur_mci_id['MINI_MENTAL_SCORE'] = patient_dur_mci_id['MINI_MENTAL_SCORE']*3
# patient_dur_mci_id.drop_duplicates(subset=['patient_id', 'durations_round(years)'],inplace=True)
# display(patient_dur_mci_id[patient_dur_mci_id.duplicated(subset=['patient_id', 'durations_round(years)'])]['patient_id'].unique().shape)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='durations_round(years)', values='MINI_MENTAL_SCORE')
#patient_cat_dur_id_pivot.fillna(0, inplace=True)
#patient_cat_dur_id_pivot.fillna(method='bfill' , inplace=True, axis=1)
#patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=False)
patient_cat_dur_id_pivot_col = patient_cat_dur_id_pivot # patient_cat_dur_id_pivot[patient_cat_dur_id_pivot.columns.tolist()[-2:]+patient_cat_dur_id_pivot.columns.tolist()[0:-2]]
display(patient_cat_dur_id_pivot_col.iloc[400:420])

import datetime
import numpy as np
import plotly.graph_objs as go

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_col.index.values]

duration_patients = patient_cat_dur_id_pivot_col.columns.values

mms_values = patient_cat_dur_id_pivot_col.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.1, "y":0.80, "title":"MMS_Scale: MINI_MENTAL_SCORE (MMSE(X))", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='MMSE score progression of Patients',xaxis_nticks=48,
                         xaxis={"title": "Duration(years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_mmsscore_duration_patientid_heatmap.html')


# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

duration_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.7, "y":0.50, "title":"MMS_Scale: MINI_MENTAL_SCORE (MMSE(X))", 'titleside':'right',"titlefont": {"size": 10} },
        showscale=True),
        layout=go.Layout(width=700, height=500, title='MMSE score progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Duration (years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 7}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_mmscore_duration_patientid_heatmap_sample.png", width=600, height=600, scale=3.0)
py.plot(fig,filename='plot_mmsscore_duration_patientid_heatmap_sample.html')


In [ ]:
def get_patient_state(l):
    # find constant
    if len(set(l))==1:
        return 'C_'+ str(l[0])
    else:
        return 'NC_'

def get_patient_st1_st2_min(l):
    t = []
    for i in range(len(l)-1):
        t.append(l[i+1]-l[i])
    if len(t)>=1:
        return min(t)    

def get_patient_st1_st2_max(l):
    t = []
    for i in range(len(l)-1):
        t.append(l[i+1]-l[i])
    if len(t)>=1:
        return max(t)    

    
#patient_com_treat_fea_raw_df['patient_state'] = #['PSYCHIATRIC','MINI_MENTAL_SCORE_CATEGORY(+MCI)'].transform(lambda x: x)
patient_com_treat_fea_raw_agg_list_df = patient_com_treat_fea_raw_df.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[1.0,2.0,3.0,4.0,5.0])\
                            .groupby(by=['patient_id']).agg(lambda x : x.tolist())


patient_com_treat_fea_raw_agg_list_df['STATE_CONSTANT']= patient_com_treat_fea_raw_agg_list_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)'].apply(get_patient_state)
patient_com_treat_fea_raw_agg_list_df['ST1_ST2_MIN']= patient_com_treat_fea_raw_agg_list_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)'].apply(get_patient_st1_st2_min)
patient_com_treat_fea_raw_agg_list_df['ST1_ST2_MAX']= patient_com_treat_fea_raw_agg_list_df['MINI_MENTAL_SCORE_CATEGORY(+MCI)'].apply(get_patient_st1_st2_max)

display(patient_com_treat_fea_raw_agg_list_df.head(15))

In [ ]:
# Non Constant Patients

patient_com_treat_fea_raw_df['durations_round(years)']= round(patient_com_treat_fea_raw_df['durations(years)'], 4)
#patient_com_treat_fea_raw_df['durations_round(years)'] = patient_com_treat_fea_raw_df['durations(years)']

# Filter Non Constant Patients (Normal and Severe)
filter_df = patient_com_treat_fea_raw_agg_list_df[~(patient_com_treat_fea_raw_agg_list_df['STATE_CONSTANT'].isin(['C_1.0', 'C_5.0', 'C_nan']))]
display(filter_df[filter_df['ST1_ST2_MAX']>=2].shape)
display(filter_df[filter_df['ST1_ST2_MIN']<=-2].shape)
display(filter_df.shape)
#display(patient_com_treat_fea_raw_df[filter_df.index])
patient_dur_mci_id = patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['patient_id'].isin(filter_df.index)][['patient_id', 'MINI_MENTAL_SCORE_CATEGORY(+MCI)', 'durations_round(years)']]
display(patient_dur_mci_id.shape)
#patient_dur_mci_id.drop_duplicates(subset=['patient_id', 'durations_round(years)'],inplace=True)
#display(patient_dur_mci_id[patient_dur_mci_id.duplicated(subset=['patient_id', 'durations_round(years)'])]['patient_id'].unique().shape)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='durations_round(years)', values='MINI_MENTAL_SCORE_CATEGORY(+MCI)')
display(patient_cat_dur_id_pivot.shape)
#patient_cat_dur_id_pivot.fillna(0, inplace=True)
#patient_cat_dur_id_pivot.fillna(method='bfill' , inplace=True, axis=1)
patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20.0,40.0,60.0,80.0,100.0],inplace=True)
#patient_cat_dur_id_pivot = patient_cat_dur_id_pivot.astype('float64')
patient_cat_dur_id_pivot.interpolate(method='linear', axis=1, limit_area='inside',  inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=False)
patient_cat_dur_id_pivot.dropna(axis=0, how='all', inplace=True)
# patient_cat_dur_id_pivot[-1]=0.0
# patient_cat_dur_id_pivot[-2]=50.0
patient_cat_dur_id_pivot_col = patient_cat_dur_id_pivot # [patient_cat_dur_id_pivot.columns.tolist()[-2:]+patient_cat_dur_id_pivot.columns.tolist()[0:-2]]
display(patient_cat_dur_id_pivot_col.iloc[400:420])

import datetime
import numpy as np
import plotly.graph_objs as go

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_col.index.values]

duration_patients = patient_cat_dur_id_pivot_col.columns.values

mms_values = patient_cat_dur_id_pivot_col.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.1, "y":0.80, "title":"MINI_MENTAL_SCORE_Category_Scale_(N/A(0),Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100))", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=4000, title='MMSE progression of Patients',xaxis_nticks=48,
                         xaxis={"title": "Duration(years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


#fig.show()

py.plot(fig,filename='plot_mmscat_duration_patientid_noconstant_heatmap.html')


# Taking sample
# Taking only few patients for the plots
patient_cat_dur_id_pivot_sam = patient_cat_dur_id_pivot.sample(n=40, random_state=5)

import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot_sam.index.values]

duration_patients = patient_cat_dur_id_pivot_sam.columns.values

mms_values = patient_cat_dur_id_pivot_sam.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.7, "y":0.50, "title":"MMS_Scale: Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100)", 'titleside':'right',"titlefont": {"size": 10} },
        showscale=True),
        layout=go.Layout(width=700, height=500, title='MMSE progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Duration (years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 7}, "tickangle": -10}))


#fig.show()
fig.write_image("plot_mmscat_duration_patientid_noconstant_heatmap_sample.png", width=600, height=600, scale=3.0)
py.plot(fig,filename='plot_mmscat_duration_patientid_noconstant_heatmap_sample.html')



In [ ]:
#Categorising patient based in their progression
patient_com_treat_fea_raw_df['durations_round(years)']= round(patient_com_treat_fea_raw_df['durations(years)'], 3)
#patient_com_treat_fea_raw_df['durations_round(years)'] = patient_com_treat_fea_raw_df['durations(years)']
patient_dur_mci_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY(+MCI)', 'durations_round(years)']]
patient_dur_mci_id.drop_duplicates(subset=['patient_id', 'durations_round(years)'],inplace=True)
display(patient_dur_mci_id[patient_dur_mci_id.duplicated(subset=['patient_id', 'durations_round(years)'])]['patient_id'].unique().shape)
patient_cat_dur_id_pivot = patient_dur_mci_id.pivot(index='patient_id', columns='durations_round(years)', values='MINI_MENTAL_SCORE_CATEGORY(+MCI)')
#patient_cat_dur_id_pivot.fillna(0, inplace=True)
patient_cat_dur_id_pivot.fillna(method='bfill' , inplace=True, axis=1)
patient_cat_dur_id_pivot.replace(['Normal', 'NormalMCI','Mild', 'Moderate', 'Severe'],[20,40,60,80,100],inplace=True)
patient_cat_dur_id_pivot.sort_values(by=[i for i in patient_cat_dur_id_pivot.columns.values], inplace=True, ascending=False)
display(patient_cat_dur_id_pivot.iloc[400:420])

'''import datetime
import numpy as np
import plotly.graph_objs as go
np.random.seed(1)

patient_ids = ["P_ID:"+str(i) for i in patient_cat_dur_id_pivot.index.values]

duration_patients = patient_cat_dur_id_pivot.columns.values

mms_values = patient_cat_dur_id_pivot.values

fig = go.Figure(data=go.Heatmap(
        z=mms_values,
        x=duration_patients,
        y=patient_ids,
        colorscale='Viridis', 
        colorbar={"len":0.3, "y":0.80, "title":"MINI_MENTAL_SCORE_Category_Scale_(N/A(0),Normal(20),NormalMCI(40),Mild(60),Moderate(80),Severe(100))", 'titleside':'right'},
        showscale=True),
        layout=go.Layout(width=1300, height=6000, title='MMSE progression of Patients',xaxis_nticks=37,
                         xaxis={"title": "Duration(years)"},
                         yaxis={"title": "Patient_ids", "tickvals":patient_ids, "ticktext":patient_ids,"tickfont": {"size": 6}, "tickangle": -10}))


fig.show()

py.plot(fig,filename='plot_mms_cat_patients_train_heatmap.html')'''

In [ ]:
patient_dur_mci_id_normal = patient_dur_mci_id[patient_dur_mci_id.columns[(patient_dur_mci_id.iloc[0:1,0:]=='Normal').values.flatten()]]
fig = go.Figure([{
    'x': patient_dur_mci_id_normal[col].dropna().index,
    'y': patient_dur_mci_id_normal[col].dropna(),
    'name': col
}  for col in patient_dur_mci_id_normal.columns])
#py.iplot()

py.plot(fig,filename='plot_mms_cat_frst_last_duration_normal.html')

In [ ]:
#patient_com_treat_fea_raw_df['durations(years)'] = 
# patient_com_treat_fea_raw_df.drop(columns=['mms_cat_containing_normal_mci_frst', 'mms_cat_containing_normal_mci_lst'], inplace=True)
patient_com_treat_fea_raw_df['mms_cat_containing_normal_mci_frst']=patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['mms_cat_containing_normal_mci'].transform(lambda x : x.iloc[0])
patient_com_treat_fea_raw_df['mms_cat_containing_normal_mci_lst']=patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['mms_cat_containing_normal_mci'].transform(lambda x : x.iloc[-1])
patient_com_treat_fea_raw_df['mms_cat_frst_last'] = patient_com_treat_fea_raw_df[['mms_cat_containing_normal_mci_frst', 'mms_cat_containing_normal_mci_lst']].agg(lambda x : str(x[0])+'_'+str(x[1]), axis=1)
data_mms_cat_frst_last = patient_com_treat_fea_raw_df[['patient_id', 'mms_cat_frst_last', 'mms_cat_containing_normal_mci_frst']][patient_com_treat_fea_raw_df.mms_cat_frst_last != 'nan_nan'].drop_duplicates()
data_mms_cat_frst_last.sort_values('mms_cat_frst_last', inplace=True)
fig = px.histogram(data_mms_cat_frst_last, x="mms_cat_frst_last")
fig.show()
py.plot(fig,filename='plot_mms_cat_frst_last.html')

In [ ]:

l= ['Categories']+data_mms_cat_frst_last.mms_cat_frst_last.unique().tolist() + data_mms_cat_frst_last.mms_cat_containing_normal_mci_frst.unique().tolist()
p = [''] + [i.split('_')[0] for i in data_mms_cat_frst_last.mms_cat_frst_last.unique()] + ['Categories' for i in data_mms_cat_frst_last.mms_cat_containing_normal_mci_frst.unique()]
mms_cat_frst_last_count = dict(data_mms_cat_frst_last.mms_cat_frst_last.value_counts())
mms_cat_containing_normal_mci_frst_count = dict(data_mms_cat_frst_last.mms_cat_containing_normal_mci_frst.value_counts())
mms_cat_fl_dict_total = {**mms_cat_frst_last_count, **mms_cat_containing_normal_mci_frst_count}
v = [sum(mms_cat_frst_last_count.values())] + [mms_cat_fl_dict_total[i] for i in l if i in mms_cat_fl_dict_total]
#v = [(i,i/sum(mms_cat_frst_last_count.values())*100) for i in v ]
print (l, len(l))
print (p, len(p))
print (v, len(v))

fig =go.Figure(go.Sunburst(
    labels=l,
    parents=p,
    values= v,
    hovertext=[(str(j)+' '+'Count: '+str(i), 'Percentage: '+str(i/sum(mms_cat_frst_last_count.values())*100)) for i,j in zip(v,l)],
    hoverinfo="text",
    branchvalues="total",
))

fig.update_layout(title=go.layout.Title(text='Figure'), margin = dict(t=100, l=0, r=100, b=0))
py.plot(fig,filename='plot_mms_cat_frst_last_sunbur.html')
fig.show()

In [ ]:
# plot MMSE Category VS Episode Number for each patient   # lets select patient based on his intial condition
# sort categorical data 
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=['Normal', 'Mild', 'Moderate', 'Severe'], ordered=True)

patient_cat_line_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY', 'lineNumber']]
display(patient_cat_line_id.head(5))
patient_cat_line_id['MINI_MENTAL_SCORE_CATEGORY'] = patient_cat_line_id['MINI_MENTAL_SCORE_CATEGORY'].astype(cat_type)
patient_cat_line_id.sort_values(by=['MINI_MENTAL_SCORE_CATEGORY'], inplace=True)
patient_cat_line_id_pivot = patient_cat_line_id.pivot(index='lineNumber', columns='patient_id', values='MINI_MENTAL_SCORE_CATEGORY')
py.iplot([{
    'x': patient_cat_line_id_pivot.index,
    'y': patient_cat_line_id_pivot[col],
    'name': col
}  for col in patient_cat_line_id_pivot.columns], filename='./patient_cat_line_id_pivot')

In [ ]:
patient_com_treat_fea_raw_df['durations(years)'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['EPISODE_DATE'].transform(lambda x: (x - x.iloc[0])/(np.timedelta64(1, 'D')*365.25))

patient_cat_dur_id = patient_com_treat_fea_raw_df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY', 'durations(years)']]
display(patient_cat_dur_id.head(5))
patient_cat_dur_id['MINI_MENTAL_SCORE_CATEGORY'] = patient_cat_dur_id['MINI_MENTAL_SCORE_CATEGORY'].astype(cat_type)
patient_cat_dur_id.sort_values(by=['MINI_MENTAL_SCORE_CATEGORY'], inplace=True)
patient_cat_dur_id_pivot = patient_cat_dur_id.pivot(index='durations(years)', columns='patient_id', values='MINI_MENTAL_SCORE_CATEGORY')

py.iplot([{
    'x': patient_cat_dur_id_pivot[col].dropna().index,
    'y': patient_cat_dur_id_pivot[col].dropna(),
    'name': col
}  for col in patient_cat_dur_id_pivot.columns], filename='./patient_cat_dur_id_pivot')


# import math 
# max_val  = math.ceil(max(patient_com_treat_fea_raw_df['durations(years)']))
# pd.cut(patient_com_treat_fea_raw_df['durations(years)'], bins=[-1,1,3,7, max_val], labels=[1,3,7, max_val]) #labels=['20s', '30s', '40s']

In [ ]:
patient_com_treat_fea_raw_df['durations(years)'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['EPISODE_DATE'].transform(lambda x: (x - x.iloc[0])/(np.timedelta64(1, 'D')*365.25))
patient_com_treat_fea_raw_df['initial_state'] = patient_com_treat_fea_raw_df.groupby(by=['patient_id'])['MINI_MENTAL_SCORE_CATEGORY'].transform(lambda x: x.iloc[0])
display(patient_com_treat_fea_raw_df.head(5))

# select patient based on intial state
def plot_duration_mmse_category(df):
    patient_cat_dur_id = df[['patient_id', 'MINI_MENTAL_SCORE_CATEGORY', 'durations(years)']]
    display(patient_cat_dur_id.head(5))
    patient_cat_dur_id['MINI_MENTAL_SCORE_CATEGORY'] = patient_cat_dur_id['MINI_MENTAL_SCORE_CATEGORY'].astype(cat_type)
    patient_cat_dur_id.sort_values(by=['MINI_MENTAL_SCORE_CATEGORY'], inplace=True)
    #patient_cat_dur_id.replace(['Normal', 'Mild', 'Moderate', 'Severe'], [0,1,2,3], inplace=True)
    #display(patient_cat_dur_id.head(10))
    patient_cat_dur_id_pivot = patient_cat_dur_id.pivot(index='durations(years)', columns='patient_id', values='MINI_MENTAL_SCORE_CATEGORY')
    #print (patient_cat_dur_id_pivot[col].dropna())
    py.iplot([{
        'x': patient_cat_dur_id_pivot[col].dropna().index,
        'y': patient_cat_dur_id_pivot[col].dropna(),
        'name': col
    }  for col in patient_cat_dur_id_pivot.columns], filename='./patient_cat_dur_id_pivot')



# for normal patients
plot_duration_mmse_category(patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['initial_state']=='Normal'])
plot_duration_mmse_category(patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['initial_state']=='Mild'])
plot_duration_mmse_category(patient_com_treat_fea_raw_df[patient_com_treat_fea_raw_df['initial_state']=='Moderate'])
